Input your Intuitionistic fuzzy preference relation matrix, criteria vs criteria in the matrix in the form of intuitionistic fuzzy sets / numbers

In [4]:
import numpy as np

# Initial IFPR matrix for criteria with respect to the overall objective
# RESEARCH PAPER CASE STUDY MATRIX
R = np.array([
    [[0.5, 0.5], [0.6, 0.2], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.2, 0.6], [0.5, 0.5], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.4, 0.2], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.4, 0.2], [0.5, 0.5]]
])

In [5]:
# RESEARHCH PAPER CASE STUDY MATRIX #2
R = np.array([
    [[0.5, 0.5], [0.2, 0.6], [0.3, 0.4], [0.6, 0.2]],
    [[0.6, 0.2], [0.5, 0.5], [0.5, 0.4], [0.6, 0.4]],
    [[0.4, 0.3], [0.4, 0.5], [0.5, 0.5], [0.3, 0.2]],
    [[0.2, 0.6], [0.4, 0.6], [0.2, 0.3], [0.5, 0.5]]
])

In [6]:
# SOFTWARE REQUIREMENT CRITERIA MATRIX 
R = np.array([
    [[0.5, 0.3], [0.4, 0.5], [0.2, 0.7]],
    [[0.5, 0.4], [0.5, 0.3], [0.3, 0.6]],
    [[0.7, 0.2], [0.6, 0.3], [0.5, 0.3]]
])

In [7]:
R= np.array([
[[0.5, 0.3], [0.2, 0.6], [0.1, 0.8]],
[[0.6, 0.2], [0.5, 0.3], [0.3, 0.6]], # ACTUAL IFPR MATRIX
[[0.8, 0.1], [0.6, 0.3], [0.5, 0.3]]
])

In [8]:
#test case requirement matrix IFPR 
R = np.array([
    # FP_count      Complexity     Time          Cost          R_Priority
    [[0.5, 0.5],   [0.7, 0.2],   [0.6, 0.3],   [0.8, 0.1],   [0.7, 0.2]],  # FP_count
    [[0.2, 0.7],   [0.5, 0.5],   [0.6, 0.3],   [0.7, 0.2],   [0.6, 0.3]],  # Complexity
    [[0.3, 0.6],   [0.3, 0.6],   [0.5, 0.5],   [0.6, 0.3],   [0.5, 0.4]],  # Time
    [[0.1, 0.8],   [0.2, 0.7],   [0.3, 0.6],   [0.5, 0.5],   [0.4, 0.5]],  # Cost
    [[0.2, 0.7],   [0.3, 0.6],   [0.4, 0.5],   [0.5, 0.4],   [0.5, 0.5]]   # R_Priority
])

In [13]:
def construct_perfect_multiplicative_consistent_ifpr(R):
    """
    Constructs the Perfect Multiplicative Consistent IFPR (R_bar) from the original IFPR R.
    
    For i < k, if we define the chain over t = i+1, ..., k-1, then:
    
       μ̃(i,k) = { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) } /
                  { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) + [∏ₜ ( (1-μ(i,t)) * (1-μ(t,k)) )]^(exponent) }
    
       ν̃(i,k) is defined similarly using ν.
    
    The exponent is chosen as follows:
      - If chain length = (k - i - 1) equals 1, exponent = 1.
      - If chain length > 1, exponent = 0.5 (i.e. take square root).
    
    For adjacent indices (k = i+1), the original values are kept.
    The lower-triangular part is filled by reciprocity:
      R_bar[k,i] = (ν̃(i,k), μ̃(i,k))
    """
    n = R.shape[0]
    R_bar = np.zeros((n, n, 2), dtype=float)

    # 1. Set diagonal entries to (0.5, 0.5)
    for i in range(n):
        R_bar[i, i] = [0.5, 0.5]

    # 2. Compute upper-triangular entries
    for i in range(n):
        for k in range(i+1, n):
            if k == i + 1:
                # For adjacent indices, simply use the original value.
                R_bar[i, k] = R[i, k]
            else:
                exponent = 1.0 / (k - i - 1)
                # Compute chain-products for membership
                prod_mu = 1.0
                prod_1mu = 1.0
                for t in range(i+1, k):
                    prod_mu   *= (R[i, t, 0] * R[t, k, 0])
                    prod_1mu  *= ((1 - R[i, t, 0]) * (1 - R[t, k, 0]))
                left_mu  = prod_mu ** exponent
                right_mu = prod_1mu ** exponent
                denom_mu = left_mu + right_mu
                new_mu = left_mu / denom_mu if denom_mu != 0 else 0.5

                # Compute chain-products for non-membership
                prod_nu = 1.0
                prod_1nu = 1.0
                for t in range(i+1, k):
                    prod_nu   *= (R[i, t, 1] * R[t, k, 1])
                    prod_1nu  *= ((1 - R[i, t, 1]) * (1 - R[t, k, 1]))
                left_nu  = prod_nu ** exponent
                right_nu = prod_1nu ** exponent
                denom_nu = left_nu + right_nu
                new_nu = left_nu / denom_nu if denom_nu != 0 else 0.5

                R_bar[i, k] = [new_mu, new_nu]

    # 3. Fill the lower-triangular part using reciprocity:
    # If R_bar[i,k] = (μ, ν), then set R_bar[k,i] = (ν, μ).
    for i in range(n):
        for k in range(i+1, n):
            mu_val, nu_val = R_bar[i, k]
            R_bar[k, i] = [nu_val, mu_val]

    return R_bar

In [14]:
# Construct the Perfect Multiplicative Consistent IFPR
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)

# Print results with better formatting
np.set_printoptions(precision=4, suppress=True)
print("Original R (upper triangle):")
print(R[..., 0], "\n", R[..., 1], "\n")  # Print membership (μ) and non-membership (ν) matrices

print("Perfect Multiplicative Consistent IFPR (R_bar):")

# Dynamically determine the size of the matrix
n = R.shape[0]  # Number of rows (and columns, since it's an N x N matrix)

# Loop through each row and column dynamically
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_bar[i, k, 0]
        nu_val = R_bar[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print(f"Row {i+1}:", "\t".join(row_str))

Original R (upper triangle):
[[1.     0.301  0.25   0.699  0.4771]
 [0.599  1.     0.25   0.699  0.4771]
 [0.65   0.65   1.     0.9    0.699 ]
 [0.201  0.201  0.     1.     0.25  ]
 [0.4229 0.4229 0.201  0.65   1.    ]] 
 [[0.     0.599  0.65   0.201  0.4229]
 [0.301  0.     0.65   0.201  0.4229]
 [0.25   0.25   0.     0.     0.201 ]
 [0.699  0.699  0.9    0.     0.65  ]
 [0.4771 0.4771 0.699  0.25   0.    ]] 

Perfect Multiplicative Consistent IFPR (R_bar):
Row 1: (0.5000, 0.5000)	(0.3010, 0.5990)	(0.1255, 0.7350)	(0.6340, 0.0000)	(0.3817, 0.3829)
Row 2: (0.5990, 0.3010)	(0.5000, 0.5000)	(0.2500, 0.6500)	(0.7500, 0.0000)	(0.4363, 0.3185)
Row 3: (0.7350, 0.1255)	(0.6500, 0.2500)	(0.5000, 0.5000)	(0.9000, 0.0000)	(0.7500, 0.0000)
Row 4: (0.0000, 0.6340)	(0.0000, 0.7500)	(0.0000, 0.9000)	(0.5000, 0.5000)	(0.2500, 0.6500)
Row 5: (0.3829, 0.3817)	(0.3185, 0.4363)	(0.0000, 0.7500)	(0.6500, 0.2500)	(0.5000, 0.5000)


In [15]:
def check_consistency(R, R_bar, tau=0.1):
    """
    Computes the distance d between the original IFPR matrix R and 
    the perfectly multiplicative consistent IFPR matrix R_bar using:
    
      d(R, R_bar) = (1 / ((n-1) * (n-2))) * sum_{i=1}^{n} sum_{k=i+1}^{n} 
                     [|mu_bar(i,k) - mu(i,k)| + |nu_bar(i,k) - nu(i,k)| + |pi_bar(i,k) - pi(i,k)|],
                     
    where for each entry,
      pi(i,k) = 1 - mu(i,k) - nu(i,k) and
      pi_bar(i,k) = 1 - mu_bar(i,k) - nu_bar(i,k).
    
    Parameters:
      R      : Original IFPR matrix (n x n x 2 numpy array)
      R_bar  : Consistent IFPR matrix (n x n x 2 numpy array)
      tau    : Threshold for acceptable consistency (default: 0.1)
    
    Returns:
      (bool) True if d < tau, else False.
    """
    n = R.shape[0]
    total_diff = 0.0
    
    # Loop over the upper triangular indices (i < k)
    for i in range(n):
        for k in range(i+1, n):
            mu = R[i, k, 0]
            nu = R[i, k, 1]
            pi = 1 - mu - nu
            
            mu_bar = R_bar[i, k, 0]
            nu_bar = R_bar[i, k, 1]
            pi_bar = 1 - mu_bar - nu_bar
            
            diff = abs(mu_bar - mu) + abs(nu_bar - nu) + abs(pi_bar - pi)
            total_diff += diff
    
    # Use normalization factor as in the paper: (n-1)*(n-2) or 2*(n-1)(n-2) idk man
    denominator = (n - 1) * (n - 2)
    d = total_diff / denominator
    
    print("Distance d(R, R_bar) =", d)
    return d < tau

In [16]:
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    
# Check consistency:
consistent = check_consistency(R, R_bar, tau=0.1)
if consistent:
    print("The IFPR is consistent (d < 0.1).")
else:
    print("The IFPR is NOT consistent (d >= 0.1).")

Distance d(R, R_bar) = 0.17885243662599595
The IFPR is NOT consistent (d >= 0.1).


In [17]:
def repair_ifahp_algorithm_2(R, sigma=0.5, tau=0.1, max_iter=100):
    """
    Repairs the IFPR R using the fused intuitionistic preference algorithm (Algorithm 2)
    until the consistency measure d(R, R_bar) < tau.
    
    The update rules for off-diagonal entries are:
    
      μ^(p+1)(i,k) = [ (μ^(p)(i,k))^(1-σ) * (μ̄(i,k))^σ ] /
                      { (μ^(p)(i,k))^(1-σ) * (μ̄(i,k))^σ + (1-μ^(p)(i,k))^(1-σ) * (1-μ̄(i,k))^σ }
    
      ν^(p+1)(i,k) = [ (ν^(p)(i,k))^(1-σ) * (ν̄(i,k))^σ ] /
                      { (ν^(p)(i,k))^(1-σ) * (ν̄(i,k))^σ + (1-ν^(p)(i,k))^(1-σ) * (1-ν̄(i,k))^σ }
    
    Diagonal entries are fixed to (0.5, 0.5) and the lower-triangular part is
    updated via reciprocity.
    """
    # First, compute the perfect IFPR matrix.
    R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    print("Initial consistency check:")
    if check_consistency(R, R_bar, tau):
        print("R is already consistent with R_bar; no repair needed.")
        return R

    # Start the iterative repair process.
    R_current = R.copy()
    n = R_current.shape[0]
    
    for p in range(max_iter):
        R_next = np.copy(R_current)
        
        # Update all off-diagonal entries using the ratio-based formulas.
        for i in range(n):
            for k in range(n):
                if i != k:
                    mu_p = R_current[i, k, 0]
                    nu_p = R_current[i, k, 1]
                    mu_bar = R_bar[i, k, 0]
                    nu_bar = R_bar[i, k, 1]
                    
                    # Membership update:
                    num_mu = (mu_p ** (1 - sigma)) * (mu_bar ** sigma)
                    den_mu = num_mu + ((1 - mu_p) ** (1 - sigma)) * ((1 - mu_bar) ** sigma)
                    mu_next = num_mu / den_mu if den_mu != 0 else 0.5
                    
                    # Non-membership update:
                    num_nu = (nu_p ** (1 - sigma)) * (nu_bar ** sigma)
                    den_nu = num_nu + ((1 - nu_p) ** (1 - sigma)) * ((1 - nu_bar) ** sigma)
                    nu_next = num_nu / den_nu if den_nu != 0 else 0.5
                    
                    R_next[i, k, 0] = mu_next
                    R_next[i, k, 1] = nu_next
        
        # Enforce the diagonal to remain (0.5, 0.5)
        for i in range(n):
            R_next[i, i] = [0.5, 0.5]
        
        # Enforce reciprocity for the lower-triangular part:
        for i in range(n):
            for k in range(i+1, n):
                mu_val, nu_val = R_next[i, k]
                R_next[k, i] = [nu_val, mu_val]
        
        print(f"Iteration {p+1} consistency check:")
        if check_consistency(R_next, R_bar, tau):
            print(f"Repair successful after {p+1} iterations.")
            return R_next
        
        R_current = R_next
    
    print("Max iterations reached; returning final repaired matrix.")
    return R_current

In [18]:
R_repaired = repair_ifahp_algorithm_2(R, sigma=0.8, tau=0.1, max_iter=100)
    
# --- Print the repaired IFPR matrix ---
print("\nRepaired IFPR matrix:")
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_repaired[i, k, 0]
        nu_val = R_repaired[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print("Row", i+1, ":", "\t".join(row_str))

Initial consistency check:
Distance d(R, R_bar) = 0.17885243662599595
Iteration 1 consistency check:
Distance d(R, R_bar) = 0.017846566909744628
Repair successful after 1 iterations.

Repaired IFPR matrix:
Row 1 : (0.5000, 0.5000)	(0.3010, 0.5990)	(0.1452, 0.7191)	(0.6475, 0.0000)	(0.4003, 0.3908)
Row 2 : (0.5990, 0.3010)	(0.5000, 0.5000)	(0.2500, 0.6500)	(0.7403, 0.0000)	(0.4444, 0.3383)
Row 3 : (0.7191, 0.1452)	(0.6500, 0.2500)	(0.5000, 0.5000)	(0.9000, 0.0000)	(0.7403, 0.0000)
Row 4 : (0.0000, 0.6475)	(0.0000, 0.7403)	(0.0000, 0.9000)	(0.5000, 0.5000)	(0.2500, 0.6500)
Row 5 : (0.3908, 0.4003)	(0.3383, 0.4444)	(0.0000, 0.7403)	(0.6500, 0.2500)	(0.5000, 0.5000)


In [19]:
def compute_criterion_weights(R):
    """
    Computes the intuitionistic fuzzy weights ω_i = (μ_i, ν_i) for each row i
    using Formula (26).
    """
    n = R.shape[0]
    weights = np.zeros((n, 2), dtype=float)
    
    # Calculate the denominators first (they are the same for all weights)
    sum_all_mu = 0.0
    sum_all_one_minus_nu = 0.0
    
    for i in range(n):
        for k in range(n):
            sum_all_mu += R[i, k, 0]  # Sum of all μ(i,k)
            sum_all_one_minus_nu += (1.0 - R[i, k, 1])  # Sum of all (1-ν(i,k))
    
    # Now calculate weights for each criterion
    for i in range(n):
        sum_mu_i = sum(R[i, k, 0] for k in range(n))  # Sum of μ(i,k) for row i
        sum_one_minus_nu_i = sum(1.0 - R[i, k, 1] for k in range(n))  # Sum of (1-ν(i,k)) for row i
        
        # Calculate weights according to formula 26
        if sum_all_one_minus_nu <= 1e-15 or sum_all_mu <= 1e-15:
            w_mu = 0.5
            w_nu = 0.5
        else:
            w_mu = sum_mu_i / sum_all_one_minus_nu
            w_nu = 1.0 - (sum_one_minus_nu_i / sum_all_mu)
        
        weights[i, 0] = w_mu
        weights[i, 1] = w_nu
    
    return weights

In [20]:
weights = compute_criterion_weights(R_repaired)

print("\nCriterion Weights (μ, ν):")
for i, (mu_w, nu_w) in enumerate(weights, start=1):
    print(f"ω_{i} = ({mu_w:.4f}, {nu_w:.4f})")


Criterion Weights (μ, ν):
ω_1 = (0.1391, 0.7383)
ω_2 = (0.1768, 0.6990)
ω_3 = (0.2448, 0.6152)
ω_4 = (0.0523, 0.8535)
ω_5 = (0.1311, 0.7501)


In [21]:
def convert_to_crisp_weights(weights):
    """
    Converts intuitionistic fuzzy weights to crisp weights.
    
    Args:
    - weights: np.ndarray of shape (n, 2) where weights[i, 0] is μ_i and weights[i, 1] is ν_i
    
    Returns:
    - crisp_weights: Normalized crisp weights
    """
    n = weights.shape[0]
    crisp_values = np.zeros(n)
    
    for i in range(n):
        mu = weights[i, 0]
        nu = weights[i, 1]
        
        # Score function: S(ω_i) = μ_i - ν_i
        score = mu - nu
        
        # Accuracy function: H(ω_i) = μ_i + ν_i
        accuracy = mu + nu
        
        # Combined score (this is one approach)
        crisp_values[i] = 0.5 * (score + 1) * (1 + accuracy - score)
    
    # Normalize to ensure sum is 1
    sum_crisp = np.sum(crisp_values)
    if sum_crisp > 0:
        crisp_weights = crisp_values / sum_crisp
    else:
        # Equal weights if all values are 0
        crisp_weights = np.ones(n) / n
        
    return crisp_weights

In [65]:
crisp_weights = convert_to_crisp_weights(weights)
print(crisp_weights)

[0.1972 0.2276 0.279  0.1069 0.1892]


In [23]:
import pandas as pd

# SOFTWARE REQUIREMENT PRIORITIZATION DATASET INIT
# Load the dataset 
data = pd.read_csv("excel_file/Requirements.csv")
data = data.drop(columns=['Pre-requisite', 'Test Cases'])
# Map Importance values ('H', 'M', 'L') to numerical values (4, 2.5, 1)
importance_mapping = {'H': 9, 'M': 5, 'L': 3}
data['Importance'] = data['Importance'].map(importance_mapping)

# Extract the decision matrix
decision_matrix = data[['Cost', 'Value', 'Importance']].values

# Print the decision matrix
print("Decision Matrix:")
print(decision_matrix)

Decision Matrix:
[[8 9 9]
 [3 2 9]
 [7 8 9]
 [4 3 5]
 [6 7 9]
 [7 8 9]
 [6 3 3]
 [5 4 3]
 [7 1 3]
 [7 3 3]
 [5 6 5]
 [6 4 5]
 [4 3 9]
 [5 7 5]
 [7 2 3]
 [7 6 5]
 [3 2 3]
 [4 5 5]
 [5 4 5]
 [5 4 5]
 [6 5 5]
 [6 6 5]
 [8 6 5]
 [7 5 5]
 [5 9 9]
 [2 6 5]
 [6 9 9]
 [7 8 9]
 [7 4 5]
 [4 8 5]
 [6 6 5]
 [8 5 3]
 [4 9 9]
 [6 8 5]
 [3 5 3]
 [4 8 9]
 [2 4 5]
 [5 8 9]
 [5 8 9]
 [7 7 5]
 [4 8 5]
 [2 8 5]
 [1 7 9]
 [5 6 5]
 [6 4 5]
 [5 9 5]
 [3 8 9]
 [6 7 9]
 [5 8 9]
 [5 7 9]
 [6 6 5]
 [4 8 9]
 [6 5 5]
 [5 3 5]
 [6 4 3]
 [2 7 5]
 [3 6 9]
 [5 7 9]
 [6 8 9]
 [6 7 5]
 [7 5 3]
 [5 7 9]
 [4 3 9]
 [3 6 9]
 [5 3 9]
 [4 3 9]
 [6 4 9]
 [3 3 9]
 [6 4 3]
 [3 1 3]
 [5 3 5]
 [4 6 9]
 [6 6 5]
 [5 7 5]
 [4 4 5]
 [3 4 5]
 [7 5 5]
 [4 3 9]
 [5 7 9]
 [3 3 3]
 [4 2 5]
 [3 2 3]
 [4 6 9]
 [5 5 9]
 [5 5 9]
 [6 8 5]
 [4 7 9]
 [5 6 9]
 [4 5 5]
 [3 7 9]
 [4 6 5]
 [6 9 9]
 [5 5 5]
 [7 8 9]
 [8 8 5]
 [6 4 9]
 [7 7 3]
 [4 8 5]
 [4 5 9]
 [5 4 9]]


In [24]:
# TEST-CASE REQUIREMENT PRIORITIZATION DATASET INIT
data = pd.read_csv("excel_file/Testing_carlease.csv")
data['FP_count'] = data['FP'].apply(lambda x: len(str(x).split(',')))
data.set_index("B_Req", inplace=True)
# Define criteria columns (excludes B_Req which is just an identifier)
criteria_columns = ['FP_count', 'Complexity', 'Time', 'Cost', 'R_Priority']
decision_matrix = data[criteria_columns].values

In [25]:
print(decision_matrix)

[[  3.    3.    4.   60.   94. ]
 [  3.    3.    4.   60.  197. ]
 [  4.    3.    5.   75.  163. ]
 ...
 [  4.    1.    5.   25.  207. ]
 [  1.    3.    1.5  22.5 168. ]
 [  1.    1.    1.5   7.5 102. ]]


In [26]:
import numpy as np
import pandas as pd

# Create the stakeholder decision matrix from Table 2
# Rows: Stakeholders S1-S5
# Columns: Requirements r1-r20
stakeholder_values = np.array([
    [4, 2, 1, 2, 5, 5, 2, 4, 4, 4, 2, 3, 4, 2, 4, 4, 4, 1, 3, 2],  # S1
    [4, 4, 2, 2, 4, 5, 1, 4, 4, 5, 2, 4, 2, 5, 4, 2, 3, 2, 4, 2],  # S2
    [5, 3, 3, 2, 4, 5, 2, 4, 4, 4, 2, 4, 1, 5, 4, 2, 2, 3, 3, 2],  # S3
    [4, 5, 2, 3, 3, 4, 2, 4, 2, 3, 5, 2, 3, 2, 4, 3, 5, 4, 3, 2],  # S4
    [5, 4, 2, 4, 5, 4, 2, 4, 5, 2, 4, 5, 3, 4, 4, 1, 1, 2, 4, 1]   # S5
])

# Create a DataFrame for better visualization
stakeholders = ["S1", "S2", "S3", "S4", "S5"]
requirements = [f"r{i}" for i in range(1, 21)]
df_stakeholder_values = pd.DataFrame(stakeholder_values, index=stakeholders, columns=requirements)

print("Stakeholder Values DataFrame (Original Decision Matrix):")
print(df_stakeholder_values)

# Option 1: Use the matrix as-is (5×20) with stakeholders as rows and requirements as columns
decision_matrix = stakeholder_values
print("\nDecision Matrix (5×20):")
print(decision_matrix)

decision_matrix_transposed = stakeholder_values.T  # Transpose
print("\nTransposed Decision Matrix (20×5):")
print(decision_matrix_transposed)

decision_matrix = decision_matrix_transposed

Stakeholder Values DataFrame (Original Decision Matrix):
    r1  r2  r3  r4  r5  r6  r7  r8  r9  r10  r11  r12  r13  r14  r15  r16  \
S1   4   2   1   2   5   5   2   4   4    4    2    3    4    2    4    4   
S2   4   4   2   2   4   5   1   4   4    5    2    4    2    5    4    2   
S3   5   3   3   2   4   5   2   4   4    4    2    4    1    5    4    2   
S4   4   5   2   3   3   4   2   4   2    3    5    2    3    2    4    3   
S5   5   4   2   4   5   4   2   4   5    2    4    5    3    4    4    1   

    r17  r18  r19  r20  
S1    4    1    3    2  
S2    3    2    4    2  
S3    2    3    3    2  
S4    5    4    3    2  
S5    1    2    4    1  

Decision Matrix (5×20):
[[4 2 1 2 5 5 2 4 4 4 2 3 4 2 4 4 4 1 3 2]
 [4 4 2 2 4 5 1 4 4 5 2 4 2 5 4 2 3 2 4 2]
 [5 3 3 2 4 5 2 4 4 4 2 4 1 5 4 2 2 3 3 2]
 [4 5 2 3 3 4 2 4 2 3 5 2 3 2 4 3 5 4 3 2]
 [5 4 2 4 5 4 2 4 5 2 4 5 3 4 4 1 1 2 4 1]]

Transposed Decision Matrix (20×5):
[[4 4 5 4 5]
 [2 4 3 5 4]
 [1 2 3 2 2]
 [2 2 2 3 4]
 

In [27]:
def determine_best_and_worst(decision_matrix, criteria_types):
    """
    Determines the best and worst values for each criterion.
    Args:
    - decision_matrix: A numpy array of shape (m, n), where m is the number of alternatives and n is the number of criteria.
    - criteria_types: A list indicating the type of each criterion ('max' for beneficial, 'min' for non-beneficial).
    Returns:
    - f_star: The best values for each criterion.
    - f_minus: The worst values for each criterion.
    """
    f_star = []
    f_minus = []

    for j, criterion_type in enumerate(criteria_types):
        if criterion_type == 'max':  # Beneficial criterion
            f_star.append(np.max(decision_matrix[:, j]))
            f_minus.append(np.min(decision_matrix[:, j]))
        elif criterion_type == 'min':  # Non-beneficial criterion
            f_star.append(np.min(decision_matrix[:, j]))
            f_minus.append(np.max(decision_matrix[:, j]))

    return np.array(f_star), np.array(f_minus)

# Define criteria types
'''criteria_types = ['min', 'max', 'max']  # Cost: min, Value: max, Importance: max'''
# criteria 2 for carlease dataset
# Higher FP_count indicates broader test coverage and more comprehensive testing for the requirement.
# criteria_types = ['max', 'min', 'min', 'min', 'max']  # 'FP_count', 'Complexity', 'Time', 'Cost', 'R_Priority'

criteria_types = ['max', 'max', 'max', 'max', 'max']


# Determine best and worst values
f_star, f_minus = determine_best_and_worst(decision_matrix, criteria_types)
print("f_star, f_minus:" , f_star, f_minus)

f_star, f_minus: [5 5 5 5 5] [1 1 1 2 1]


In [28]:
def substitute_values(decision_matrix, weights, f_star, f_minus):
    """
    Substitutes each value in the decision matrix with the given formula.
    Args:
    - decision_matrix: The decision matrix.
    - weights: The weights for each criterion.
    - f_star: The best values for each criterion.
    - f_minus: The worst values for each criterion.
    Returns:
    - substituted_matrix: The substituted decision matrix.
    """
    m, n = decision_matrix.shape
    substituted_matrix = np.zeros_like(decision_matrix, dtype=float)

    for i in range(m):
        for j in range(n):
            numerator = f_star[j] - decision_matrix[i, j]
            denominator = f_star[j] - f_minus[j] + 1e-10
            substituted_matrix[i, j] = crisp_weights[j] * (numerator / denominator)

    return substituted_matrix

# Substitute values in the decision matrix
substituted_matrix = substitute_values(decision_matrix, crisp_weights, f_star, f_minus)
print("\nSubstituted Decision Matrix:")
print(substituted_matrix)


Substituted Decision Matrix:
[[0.0493 0.0569 0.     0.0356 0.    ]
 [0.1479 0.0569 0.1395 0.     0.0473]
 [0.1972 0.1707 0.1395 0.1069 0.1419]
 [0.1479 0.1707 0.2093 0.0713 0.0473]
 [0.     0.0569 0.0698 0.0713 0.    ]
 [0.     0.     0.     0.0356 0.0473]
 [0.1479 0.2276 0.2093 0.1069 0.1419]
 [0.0493 0.0569 0.0698 0.0356 0.0473]
 [0.0493 0.0569 0.0698 0.1069 0.    ]
 [0.0493 0.     0.0698 0.0713 0.1419]
 [0.1479 0.1707 0.2093 0.     0.0473]
 [0.0986 0.0569 0.0698 0.1069 0.    ]
 [0.0493 0.1707 0.279  0.0713 0.0946]
 [0.1479 0.     0.     0.1069 0.0473]
 [0.0493 0.0569 0.0698 0.0356 0.0473]
 [0.0493 0.1707 0.2093 0.0713 0.1892]
 [0.0493 0.1138 0.2093 0.     0.1892]
 [0.1972 0.1707 0.1395 0.0356 0.1419]
 [0.0986 0.0569 0.1395 0.0713 0.0473]
 [0.1479 0.1707 0.2093 0.1069 0.1892]]


In [29]:
def calculate_s(substituted_matrix):
    """
    Calculates the utility measure (S_i) for each alternative.
    Args:
    - substituted_matrix: The substituted decision matrix.
    Returns:
    - S: The utility measure for each alternative.
    """
    S = np.sum(substituted_matrix, axis=1)
    return S

# Calculate S_i
S = calculate_s(substituted_matrix)
print("\nUtility Measure (S):", S)


Utility Measure (S): [0.1418 0.3916 0.7563 0.6465 0.1979 0.0829 0.8336 0.2589 0.2829 0.3323
 0.5752 0.3322 0.6649 0.3021 0.2589 0.6898 0.5616 0.685  0.4136 0.824 ]


In [30]:
def calculate_r(substituted_matrix):
    """
    Calculates the regret measure (R_i) for each alternative.
    Args:
    - substituted_matrix: The substituted decision matrix.
    Returns:
    - R: The regret measure for each alternative.
    """
    R = np.max(substituted_matrix, axis=1)
    return R

# Calculate R_i
R = calculate_r(substituted_matrix)
print("\nRegret Measure (R):", R)
len(R)


Regret Measure (R): [0.0569 0.1479 0.1972 0.2093 0.0713 0.0473 0.2276 0.0698 0.1069 0.1419
 0.2093 0.1069 0.279  0.1479 0.0698 0.2093 0.2093 0.1972 0.1395 0.2093]


20

In [31]:
def calculate_q(S, R, S_star, S_minus, R_star, R_minus, v=0.5):
    """
    Calculates the compromise solution (Q_i) for each alternative.
    Args:
    - S: The utility measure for each alternative.
    - R: The regret measure for each alternative.
    - S_star, S_minus, R_star, R_minus: Best and worst values for S_i and R_i.
    - v: Strategy weight (default is 0.5).
    Returns:
    - Q: The compromise solution for each alternative.
    """
    m = len(S)
    Q = np.zeros(m)

    for i in range(m):
        s_term = (S[i] - S_star) / (S_minus - S_star + 1e-10)
        r_term = (R[i] - R_star) / (R_minus - R_star + 1e-10)
        Q[i] = v * s_term + (1 - v) * r_term

    return Q

# Calculate S*, R*, S-, R-
S_star = np.min(S)
S_minus = np.max(S)
R_star = np.min(R)
R_minus = np.max(R)

# Calculate Q_i
Q = calculate_q(S, R, S_star, S_minus, R_star, R_minus)

print("\nCompromise Solution (Q):\n", Q)


Compromise Solution (Q):
 [0.0599 0.4227 0.772  0.7248 0.1283 0.     0.8891 0.1656 0.2618 0.3702
 0.6773 0.2946 0.8876 0.3631 0.1656 0.7537 0.6683 0.7245 0.4192 0.8431]


In [32]:
def rank_alternatives(Q, S, R):
    """
    Ranks alternatives based on Q, S, and R.
    Args:
    - Q: The compromise solution for each alternative.
    - S: The utility measure for each alternative.
    - R: The regret measure for each alternative.
    Returns:
    - rankings: A list of tuples containing the alternative index and its Q, S, and R values, sorted by Q.
    """
    rankings = sorted(enumerate(zip(Q, S, R)), key=lambda x: (x[1][0], x[1][1], x[1][2]))
    return rankings

# Rank alternatives
rankings = rank_alternatives(Q, S, R)

# Print rankings
print("\nRanked Alternatives:")
for rank, (index, (q, s, r)) in enumerate(rankings, start=1):
    print(f"Rank {rank}: Alternative {index + 1}, Q = {q:.4f}, S = {s:.4f}, R = {r:.4f}")


Ranked Alternatives:
Rank 1: Alternative 6, Q = 0.0000, S = 0.0829, R = 0.0473
Rank 2: Alternative 1, Q = 0.0599, S = 0.1418, R = 0.0569
Rank 3: Alternative 5, Q = 0.1283, S = 0.1979, R = 0.0713
Rank 4: Alternative 8, Q = 0.1656, S = 0.2589, R = 0.0698
Rank 5: Alternative 15, Q = 0.1656, S = 0.2589, R = 0.0698
Rank 6: Alternative 9, Q = 0.2618, S = 0.2829, R = 0.1069
Rank 7: Alternative 12, Q = 0.2946, S = 0.3322, R = 0.1069
Rank 8: Alternative 14, Q = 0.3631, S = 0.3021, R = 0.1479
Rank 9: Alternative 10, Q = 0.3702, S = 0.3323, R = 0.1419
Rank 10: Alternative 19, Q = 0.4192, S = 0.4136, R = 0.1395
Rank 11: Alternative 2, Q = 0.4227, S = 0.3916, R = 0.1479
Rank 12: Alternative 17, Q = 0.6683, S = 0.5616, R = 0.2093
Rank 13: Alternative 11, Q = 0.6773, S = 0.5752, R = 0.2093
Rank 14: Alternative 18, Q = 0.7245, S = 0.6850, R = 0.1972
Rank 15: Alternative 4, Q = 0.7248, S = 0.6465, R = 0.2093
Rank 16: Alternative 16, Q = 0.7537, S = 0.6898, R = 0.2093
Rank 17: Alternative 3, Q = 0.7720

Condition Checking Acceptable Advantage (C1) : The difference between the compromise scores of the top two alternatives (Q(A 2 )−Q(A 1 )) must be greater than or equal to DQ, where: DQ= j−1 1​

Here, j is the total number of alternatives. Acceptable Stability (C2) : The best alternative (A 1 ) must also rank first in either the utility measure (S i​) or the regret measure (R i​). If either condition is violated, the compromise solution is extended to include adnal alternatives.

In [34]:
def check_vikor_conditions(Q, S, R, rankings):
    """
    Checks the VIKOR conditions for acceptable advantage and stability.
    
    Args:
    - Q: The compromise solution (Q_i) for each alternative.
    - S: The utility measure (S_i) for each alternative.
    - R: The regret measure (R_i) for each alternative.
    - rankings: A list of tuples containing the alternative index and its Q, S, R values, sorted by Q.
    
    Returns:
    - Dictionary with analysis results
    """
    n = len(Q)  # Total number of alternatives
    DQ = 1 / (n - 1)  # Threshold for acceptable advantage
    
    # Get ranks for all alternatives in S and R
    S_rankings = sorted(enumerate(S), key=lambda x: x[1])
    R_rankings = sorted(enumerate(R), key=lambda x: x[1])
    
    # Extract indices for alternatives that rank first in S or R
    best_in_S = S_rankings[0][0]
    best_in_R = R_rankings[0][0]
    
    # Check stability condition for the best alternative
    best_alt_overall = rankings[0][0]  # Index of the alternative with the best Q value
    stability_satisfied = (best_alt_overall == best_in_S) or (best_alt_overall == best_in_R)
    
    # Check acceptable advantage condition
    if len(rankings) > 1:
        advantage_satisfied = (rankings[1][1][0] - rankings[0][1][0]) >= DQ
    else:
        advantage_satisfied = True  # Only one alternative, so advantage is satisfied by default
    
    # Group alternatives that could be considered equivalent based on VIKOR conditions
    equiv_groups = []
    current_group = [rankings[0][0]]  # Start with the best alternative
    
    # If acceptable advantage is not satisfied, identify equivalent alternatives
    if not advantage_satisfied and len(rankings) > 1:
        i = 1
        while i < len(rankings) and (rankings[i][1][0] - rankings[0][1][0]) < DQ:
            current_group.append(rankings[i][0])
            i += 1
    
    equiv_groups.append(current_group)
    
    # Continue grouping remaining alternatives based on acceptable advantage
    if len(rankings) > 1:
        i = len(current_group)
        while i < len(rankings):
            new_group = [rankings[i][0]]
            next_idx = i + 1
            while next_idx < len(rankings) and (rankings[next_idx][1][0] - rankings[i][1][0]) < DQ:
                new_group.append(rankings[next_idx][0])
                next_idx += 1
            equiv_groups.append(new_group)
            i = next_idx
    
    # Create result dictionary with all alternatives and their groupings
    result = {
        "stability_satisfied": stability_satisfied,
        "advantage_satisfied": advantage_satisfied,
        "equivalent_groups": equiv_groups,
        "all_alternatives": rankings
    }
    
    return result

def display_vikor_results(result, Q, S, R):
    """
    Displays the VIKOR condition analysis results in a readable format.
    
    Args:
    - result: The dictionary returned by check_vikor_conditions
    - Q, S, R: The VIKOR measures for each alternative
    """
    print("VIKOR Condition Analysis:")
    print("========================\n")
    
    # Display satisfaction of conditions
    print(f"Stability condition: {'Satisfied' if result['stability_satisfied'] else 'Not satisfied'}")
    print(f"Acceptable advantage condition: {'Satisfied' if result['advantage_satisfied'] else 'Not satisfied'}")
    print()
    
    # Display equivalent groups
    print("Alternative groupings based on VIKOR conditions:")
    for i, group in enumerate(result['equivalent_groups']):
        print(f"Group {i+1}:")
        for alt_idx in group:
            # Find this alternative in the rankings to get its Q, S, R values
            for ranking in result['all_alternatives']:
                if ranking[0] == alt_idx:
                    q, s, r = ranking[1]
                    print(f"  Alternative {alt_idx + 1}: Q = {q:.4f}, S = {s:.4f}, R = {r:.4f}")
                    break
        
        if len(group) > 1:
            print(f"  These alternatives can be considered equivalent by VIKOR conditions")
        
        # Special note for first group if relevant
        if i == 0 and not result['advantage_satisfied']:
            print("  Note: The advantage condition is not satisfied for this group.")
            print("  Consider all alternatives in this group as potential compromise solutions.")
        
        print()

# Example usage:
result = check_vikor_conditions(Q, S, R, rankings)
display_vikor_results(result, Q, S, R)

VIKOR Condition Analysis:

Stability condition: Satisfied
Acceptable advantage condition: Satisfied

Alternative groupings based on VIKOR conditions:
Group 1:
  Alternative 6: Q = 0.0000, S = 0.0829, R = 0.0473

Group 2:
  Alternative 1: Q = 0.0599, S = 0.1418, R = 0.0569

Group 3:
  Alternative 5: Q = 0.1283, S = 0.1979, R = 0.0713
  Alternative 8: Q = 0.1656, S = 0.2589, R = 0.0698
  Alternative 15: Q = 0.1656, S = 0.2589, R = 0.0698
  These alternatives can be considered equivalent by VIKOR conditions

Group 4:
  Alternative 9: Q = 0.2618, S = 0.2829, R = 0.1069
  Alternative 12: Q = 0.2946, S = 0.3322, R = 0.1069
  These alternatives can be considered equivalent by VIKOR conditions

Group 5:
  Alternative 14: Q = 0.3631, S = 0.3021, R = 0.1479
  Alternative 10: Q = 0.3702, S = 0.3323, R = 0.1419
  These alternatives can be considered equivalent by VIKOR conditions

Group 6:
  Alternative 19: Q = 0.4192, S = 0.4136, R = 0.1395
  Alternative 2: Q = 0.4227, S = 0.3916, R = 0.1479
  Th